In [7]:
# crawl through the root directory and count number of files in folder GoPro
import os
import os.path
import re
import subprocess
import json
import pandas as pd

# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/' # 2023 August
# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/May_2024/May24_updated_structure/ego_camera/' # 2024 May
# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/Sep_2024/Raw/' # 2024 September

root_dir = '/standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/'

output_csv = './AAAI26_stat_outputs/AAAI26_dataset_stats.csv'


In [2]:
import os
import csv
import subprocess
import json
import re

def get_stats(root_dir, output_csv):
    count = 0
    # Open CSV file for writing
    with open(output_csv, mode='w', newline='') as csv_file:
        fieldnames = ['filename', 'duration', 'total_frames']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if re.match(r'.*rgb_final\.mp4$', file, re.IGNORECASE) or re.match(r'.*rgb_partial\.mp4$', file, re.IGNORECASE):
                    # Get full path of the file
                    file_path = os.path.join(root, file)
                    duration, num_frames = get_video_info(file_path) # for ffprobe
                    # duration, num_frames = get_video_info_ffprobe(file_path) # for opencv
                    
                    if duration is not None and num_frames is not None:
                        print(f"Processing {file_path}: Duration = {duration} seconds, Frames = {num_frames}")
                        
                        # Write the video info to the CSV file
                        writer.writerow({
                            'filename': file_path,
                            'duration': duration,
                            'total_frames': num_frames
                        })
                        
                    else:
                        print(f"Could not retrieve video information for {file_path}")
                    
                    count += 1
    
    return count

def get_video_info(file_path):
    # Use ffprobe to get video information
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-count_packets',
        '-show_entries', 'stream=duration,nb_frames',
        '-of', 'json',
        file_path
    ]
    try:
        print(f"Fetching info for {file_path}")
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        video_info = json.loads(result.stdout)
        

        if 'streams' in video_info and len(video_info['streams']) > 0:
            duration = float(video_info['streams'][0].get('duration', 0))
            num_frames = int(video_info['streams'][0].get('nb_frames', 0))
            return duration, num_frames
        else:
            return None, None
    except Exception as e:
        print(f"Error while fetching video info: {e}")
        return None, None




In [3]:
# number of GoPro recordings in the root directory
file_count = get_stats(root_dir, output_csv)
print(f"Processed {file_count} video files. Stats saved to {output_csv}.")


Fetching info for /standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/opvrs_15/stroke_unresponsive/0/ego/opvrs15_strokeunresponsive_0_ego_rgb_final.mp4
Processing /standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/opvrs_15/stroke_unresponsive/0/ego/opvrs15_strokeunresponsive_0_ego_rgb_final.mp4: Duration = 439.033333 seconds, Frames = 13171
Fetching info for /standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/P4/cardiac_arrest/s5/exo/P4_cardiacarrest_s5_exo_rgb_final.mp4
Processing /standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/P4/cardiac_arrest/s5/exo/P4_cardiacarrest_s5_exo_rgb_final.mp4: Duration = 67.133333 seconds, Frames = 2014
Fetching info for /standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/P4/cardiac_arrest/s5/ego/P4_cardiacarrest_s5_ego_rgb_partial.mp4
Processing /standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/P4/cardiac_arrest/s5/ego/P4_cardiacarrest_s5_ego_rgb_partial.mp4: Duration = 67.166667 seconds, Frames = 2015
Fetching info for /s

# find length scenario wise

In [8]:
final_video_stats = pd.read_csv(output_csv)


# Function to extract subject, trial, and scenario from filename
def extract_metadata(row):
    parts = row['filename'].split('/')
    subject = parts[-5]  # Example: 'P4'
    scenario = parts[-4]  # Example: 'cardiac_arrest'
    trial = parts[-3]     # Example: 's5' or '0' for numbered trials
    return pd.Series([subject, trial, scenario])

# Apply the function to add new columns to the DataFrame
final_video_stats[['subject', 'trial', 'scenario']] = final_video_stats.apply(extract_metadata, axis=1)

# Save the updated DataFrame to a new CSV file
final_video_stats.to_csv('./AAAI26_stat_outputs/updated_final_video_stats.csv', index=False)

In [9]:
import pandas as pd


# Function to classify subjects
def classify_subject(subject):
    if subject.startswith("P"):
        return "General Public"
    elif subject in ["ng7", "ng8", "ng9"]:
        return "Members"
    else:
        return "EMTs"

# Apply classification to add a new column for subject type
final_video_stats['subject_type'] = final_video_stats['subject'].apply(classify_subject)

# Group by subject type and scenario, then calculate the total duration
duration_summary = final_video_stats.groupby(['subject_type', 'scenario'])['duration'].sum().reset_index()

# add duration in minutes column
duration_summary['duration_min'] = duration_summary['duration'] / 60

# add duration in hours column
duration_summary['duration_hr'] = duration_summary['duration_min'] / 60

# Save the summary to a new CSV file (optional)
duration_summary.to_csv('./AAAI26_stat_outputs/duration_summary_by_subject_type_and_scenario.csv', index=False)

# total duration in hours
total_duration = duration_summary['duration_hr'].sum()
print(f"Total duration of all videos: {total_duration:.2f} hours")
# Print or return the summary DataFrame
print(duration_summary)


Total duration of all videos: 26.29 hours
     subject_type               scenario      duration  duration_min  \
0            EMTs         cardiac_arrest  18333.466657    305.557778   
1            EMTs       cardiac_scenario   2735.666666     45.594444   
2            EMTs             chest_pain  10614.966664    176.916111   
3            EMTs  chest_pain_noncardiac   1210.566666     20.176111   
4            EMTs                 stroke  39815.366665    663.589444   
5            EMTs    stroke_hypoglycemic   2105.833333     35.097222   
6            EMTs    stroke_unresponsive   2194.533332     36.575556   
7  General Public         cardiac_arrest  13935.366658    232.256111   
8         Members         cardiac_arrest   3694.066665     61.567778   

   duration_hr  
0     5.092630  
1     0.759907  
2     2.948602  
3     0.336269  
4    11.059824  
5     0.584954  
6     0.609593  
7     3.870935  
8     1.026130  


In [12]:
import pandas as pd


# load the updated CSV file
updated_final_video_stats = pd.read_csv('./AAAI26_stat_outputs/updated_final_video_stats.csv')

# sources and subjects

ng_subjects = ['ng1', 'ng2', 'ng3', 'ng4', 'ng5', 'ng6', 'ng7', 'ng8', 'ng9']

wars_subjects = ['wa1', 'wa2', 'wa3', 'wa4', 'wa5', 'wa6', 'wa7', 'wa8', 'wa9']

medical_school_subjects = ['ms1', 'ms2', 'ms3', 'ms4', 'ms5', 'ms6', 'ms7', 'ms8', 'ms9']

general_public_subjects = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 
                           'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 
                           'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25']

cars_subjects = ['cars_1', 'cars_2', 'cars_3', 'cars_4', 'cars_5', 'cars_6',
                 'cars_7', 'cars_8', 'cars_9', 'cars_10', 'cars_11', 'cars_12',
                 'cars_13', 'cars_14', 'cars_15', 'cars_16']

opvrs_subjects = ['opvrs_1', 'opvrs_2', 'opvrs_3', 'opvrs_4', 'opvrs_5',
                  'opvrs_6', 'opvrs_7', 'opvrs_8', 'opvrs_9', 'opvrs_10',
                  'opvrs_11', 'opvrs_12', 'opvrs_13', 'opvrs_14', 'opvrs_15']

# Function to classify subjects
def classify_subject(subject):
    if subject.startswith("P"):
        return "General Public"
    elif subject in ng_subjects:
        return "North Garden"
    elif subject in wars_subjects:
        return "WARS"
    elif subject in medical_school_subjects:
        return "Medical School"
    elif subject in cars_subjects:
        return "Charlottesville Albemarle Rescue Squad"
    elif subject in opvrs_subjects:
        return "Ocean Park Volunteer Rescue Squad"
    else:
        return "Unknown"
    
# Apply classification to add a new column for subject type
updated_final_video_stats['subject_type'] = updated_final_video_stats['subject'].apply(classify_subject)
# Group by subject type and scenario, then calculate the total duration, total subject count, and total video count
updated_final_video_stats['total_subject_count'] = updated_final_video_stats.groupby(['subject_type', 'scenario'])['subject'].transform('nunique')
updated_final_video_stats['total_video_count'] = updated_final_video_stats.groupby(['subject_type', 'scenario'])['filename'].transform('count')
updated_final_video_stats['total_duration'] = updated_final_video_stats.groupby(['subject_type', 'scenario'])['duration'].transform('sum')

# Group and aggregate
duration_summary = updated_final_video_stats.groupby(['subject_type', 'scenario']).agg(
    total_duration=('duration', 'sum'),
    total_subjects=('subject', 'nunique'),
    total_videos=('filename', 'count')
).reset_index()

# Add duration in minutes and hours
duration_summary['duration_minutes'] = duration_summary['total_duration'] / 60
duration_summary['duration_hours'] = duration_summary['total_duration'] / 3600


print(duration_summary)

# Save the summary to a new CSV file (optional)
duration_summary.to_csv('./AAAI26_stat_outputs/unique_duration_summary_by_subject_type_and_scenario.csv', index=False)




                              subject_type               scenario  \
0   Charlottesville Albemarle Rescue Squad             chest_pain   
1   Charlottesville Albemarle Rescue Squad                 stroke   
2                           General Public         cardiac_arrest   
3                           Medical School         cardiac_arrest   
4                           Medical School             chest_pain   
5                             North Garden         cardiac_arrest   
6        Ocean Park Volunteer Rescue Squad             chest_pain   
7        Ocean Park Volunteer Rescue Squad  chest_pain_noncardiac   
8        Ocean Park Volunteer Rescue Squad                 stroke   
9        Ocean Park Volunteer Rescue Squad    stroke_hypoglycemic   
10       Ocean Park Volunteer Rescue Squad    stroke_unresponsive   
11                                    WARS         cardiac_arrest   
12                                    WARS       cardiac_scenario   
13                                

In [13]:

import os
import pandas as pd

# 1) Load the updated CSV file
updated_final_video_stats = pd.read_csv('./AAAI26_stat_outputs/updated_final_video_stats.csv')

# 2) Build a per‐subject, per‐scenario summary
subject_scenario_summary = (
    updated_final_video_stats
    .groupby(['subject','scenario'])
    .agg(
        total_videos   = ('filename', 'count'),
        total_duration = ('duration', 'sum'),
    )
    .reset_index()
)

# (Optional) if you also want duration in minutes/hours:
subject_scenario_summary['duration_minutes'] = subject_scenario_summary['total_duration'] / 60
subject_scenario_summary['duration_hours']   = subject_scenario_summary['total_duration'] / 3600

# 3) Save to CSV
os.makedirs('./AAAI26_stat_outputs', exist_ok=True)
subject_scenario_summary.to_csv(
    './AAAI26_stat_outputs/summary_by_subject_and_scenario.csv',
    index=False
)

print(subject_scenario_summary)


   subject          scenario  total_videos  total_duration  duration_minutes  \
0       P0    cardiac_arrest            16     1265.899999         21.098333   
1       P1    cardiac_arrest            10      714.800000         11.913333   
2      P10    cardiac_arrest             4      234.200000          3.903333   
3      P11    cardiac_arrest            16     1072.066666         17.867778   
4      P12    cardiac_arrest            12      943.266666         15.721111   
..     ...               ...           ...             ...               ...   
72     wa1    cardiac_arrest             4     2038.500000         33.975000   
73     wa1  cardiac_scenario             8     2735.666666         45.594444   
74     wa2        chest_pain             2      407.000000          6.783333   
75     wa4        chest_pain             4     1246.700000         20.778333   
76     wa6        chest_pain             2      726.700000         12.111667   

    duration_hours  
0         0.351639

In [14]:
import pandas as pd
import os

# 1) (Re-)load your stats & classify subjects, if you haven’t already:
updated_final_video_stats = pd.read_csv('./AAAI26_stat_outputs/updated_final_video_stats.csv')

# your classify_subject() from before…
def classify_subject(subject):
    if subject.startswith("P"):
        return "General Public"
    elif subject in ng_subjects:
        return "North Garden"
    elif subject in wars_subjects:
        return "WARS"
    elif subject in medical_school_subjects:
        return "Medical School"
    elif subject in cars_subjects:
        return "Charlottesville Albemarle Rescue Squad"
    elif subject in opvrs_subjects:
        return "Ocean Park Volunteer Rescue Squad"
    else:
        return "Unknown"

updated_final_video_stats['subject_type'] = (
    updated_final_video_stats['subject'].apply(classify_subject)
)

# 2) Dedupe so each (subject, scenario, subject_type) appears only once
dedup = (
    updated_final_video_stats
    [['subject','subject_type','scenario']]
    .drop_duplicates()
)

# 3) Group & count unique subjects by scenario & category
counts = (
    dedup
    .groupby(['scenario','subject_type'])
    .agg(num_subjects=('subject','nunique'))
    .reset_index()
)

# 4) Pivot so that each scenario is a row, categories are columns
pivot = (
    counts
    .pivot(index='scenario', columns='subject_type', values='num_subjects')
    .fillna(0)
    .astype(int)
    .reset_index()
)

# 5) Save it out
os.makedirs('./AAAI26_stat_outputs', exist_ok=True)
pivot.to_csv(
    './AAAI26_stat_outputs/subjects_per_category_per_scenario.csv',
    index=False
)

print(pivot)


subject_type               scenario  Charlottesville Albemarle Rescue Squad  \
0                    cardiac_arrest                                       0   
1                  cardiac_scenario                                       0   
2                        chest_pain                                       1   
3             chest_pain_noncardiac                                       0   
4                            stroke                                      16   
5               stroke_hypoglycemic                                       0   
6               stroke_unresponsive                                       0   

subject_type  General Public  Medical School  North Garden  \
0                         16               2             9   
1                          0               0             0   
2                          0               1             0   
3                          0               0             0   
4                          0               0             